In [1]:

# ===  Setup & utilidades ===
import warnings, pathlib, random, sys
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# (Opcional) versiones para diagnóstico
try:
    import sklearn
    SKLEARN_VERSION = sklearn.__version__
except Exception:
    SKLEARN_VERSION = "no_sklearn"

# Semilla y rutas
SEED = 42
random.seed(SEED); np.random.seed(SEED)

BASE_DIR = Path("..")
DATA_DIR = BASE_DIR / "data" / "final_provider" # Ruta oficial de la plataforma

# Carpeta para artefactos (modelos, gráficos, etc.)
ARTIFACTS = BASE_DIR / "artifacts"
ARTIFACTS.mkdir(parents=True, exist_ok=True)


# Helpers
def to_datetime_safe(x):
    """Convierte a datetime; valores no parseables -> NaT."""
    return pd.to_datetime(x, errors="coerce")

def read_csv_safe(path: pathlib.Path) -> pd.DataFrame:
    """Lee CSV con verificación de existencia y tipos básicos."""
    if not path.exists():
        raise FileNotFoundError(f"No se encontró el archivo: {path}")
    df = pd.read_csv(path)
    return df

def coerce_numeric(series):
    """Convierte a numérico; errores -> NaN (útil antes de imputar)."""
    return pd.to_numeric(series, errors="coerce")

# Display amigable
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 120)

print("== Entorno listo ==")
print(f"Python: {sys.version.split()[0]} | pandas: {pd.__version__} | sklearn: {SKLEARN_VERSION}")
print(f"DATA_DIR: {DATA_DIR}")
print(f"ARTIFACTS: {ARTIFACTS.resolve()}")

== Entorno listo ==
Python: 3.12.7 | pandas: 2.2.3 | sklearn: 1.5.1
DATA_DIR: ..\data\final_provider
ARTIFACTS: C:\Users\Digitalife\interconnect-churn-prediction\artifacts


In [2]:
# === Carga de datos (contract, personal, internet, phone) ===

# Rutas de los 4 archivos esperados
contract_path = DATA_DIR / "contract.csv"
personal_path = DATA_DIR / "personal.csv"
internet_path = DATA_DIR / "internet.csv"
phone_path = DATA_DIR / "phone.csv"

# Lectura segura (usa read_csv_safe de la Celda 1)
contract = pd.read_csv(contract_path)
personal = pd.read_csv(personal_path)
internet = pd.read_csv(internet_path)
phone = pd.read_csv(phone_path)

# Resumen rápido
print("Shapes:")
print("  contract:", contract.shape)
print("  personal:", personal.shape)
print("  internet:", internet.shape)
print("  phone   :", phone.shape)

# Muestra corta para verificar columnas clave
print("\nVista rápida (contract.head()):")
display(contract.head(3))

# Comprobaciones mínimas
required_col = "customerID"
for name, df in [("contract", contract), ("personal", personal), ("internet", internet), ("phone", phone)]:
    if required_col not in df.columns:
        raise KeyError(f"Falta la columna '{required_col}' en {name}")
print("\n✅ Archivos cargados y columna 'customerID' presente en todos.")

Shapes:
  contract: (7043, 8)
  personal: (7043, 5)
  internet: (5517, 8)
  phone   : (6361, 2)

Vista rápida (contract.head()):


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15



✅ Archivos cargados y columna 'customerID' presente en todos.


In [3]:
# === Unión, fechas y target churn ===

#  Unir tablas (contract como principal)
dfs = (
    contract
    .merge(personal, on="customerID", how="left", validate="1:1")
    .merge(internet, on="customerID", how="left", validate="1:1")
    .merge(phone,    on="customerID", how="left", validate="1:1")
).copy()

print("Shape unificado:", dfs.shape)

# Parseo de fechas
dfs["BeginDate_dt"] = to_datetime_safe(dfs["BeginDate"])
dfs["EndDate_dt"]   = to_datetime_safe(dfs["EndDate"])

# Target: churn = 1 si EndDate != "No"
dfs["churn"] = (dfs["EndDate"].astype(str).str.strip().str.lower() != "no").astype(int)

# Definir fecha de predicción
# Lineamiento del supervisor: predecimos cuando existe EndDate registrado.
# Para no-churners (EndDate == "No"), usamos el snapshot del caso: 2020-02-01.
SNAPSHOT = pd.Timestamp("2020-02-01")
dfs["prediction_date"] = np.where(dfs["churn"].eq(1), dfs["EndDate_dt"], SNAPSHOT)
dfs["prediction_date"] = pd.to_datetime(dfs["prediction_date"], errors="coerce")

# Coerción numérica básica de cargos (limpieza previa a ingeniería)
for col in ["MonthlyCharges", "TotalCharges"]:
    if col in dfs.columns:
        dfs[col] = coerce_numeric(dfs[col])

# Controles rápidos
print("Fechas — BeginDate_dt nulos:", dfs["BeginDate_dt"].isna().sum())
print("Fechas — EndDate_dt   nulos:", dfs["EndDate_dt"].isna().sum())
print("Target churn distribución:")
print(dfs["churn"].value_counts(dropna=False))

# Chequeo de consistencia temporal: prediction_date debe ser >= BeginDate_dt
bad = (dfs["prediction_date"] < dfs["BeginDate_dt"]).sum()
print("Filas con prediction_date < BeginDate:", bad)

# Vista de verificación
display(dfs[["customerID","BeginDate","EndDate","BeginDate_dt","EndDate_dt","prediction_date","churn"]].head(5))

Shape unificado: (7043, 20)
Fechas — BeginDate_dt nulos: 0
Fechas — EndDate_dt   nulos: 5174
Target churn distribución:
churn
0    5174
1    1869
Name: count, dtype: int64
Filas con prediction_date < BeginDate: 0


,customerID,BeginDate,EndDate,BeginDate_dt,EndDate_dt,prediction_date,churn
0,7590-VHVEG,2020-01-01,No,2020-01-01,NaT,2020-02-01,0
1,5575-GNVDE,2017-04-01,No,2017-04-01,NaT,2020-02-01,0
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,2019-10-01,2019-12-01,2019-12-01,1
3,7795-CFOCW,2016-05-01,No,2016-05-01,NaT,2020-02-01,0
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,2019-09-01,2019-11-01,2019-11-01,1


In [4]:
# ===  Ingeniería de variables y fix de prediction_date ===

# Recalcular prediction_date de forma "segura" (sin NaT)
SNAPSHOT = pd.Timestamp("2020-02-01")
dfs["prediction_date"] = dfs["EndDate_dt"].where(dfs["churn"].eq(1), SNAPSHOT)

# Coerción numérica de cargos (por si algún valor quedó como texto)
for col in ["MonthlyCharges", "TotalCharges"]:
    if col in dfs.columns:
        dfs[col] = pd.to_numeric(dfs[col], errors="coerce")

# Tenure con respecto a prediction_date (en días y meses)
dfs["tenure_days"] = (dfs["prediction_date"] - dfs["BeginDate_dt"]).dt.days
dfs["tenure_days"] = dfs["tenure_days"].clip(lower=0)  # no negativos
dfs["tenure_months"] = (dfs["tenure_days"] / 30.44).round(2)

# Cargo mensual promedio precautorio (mitiga posible fuga de TotalCharges)
dfs["avg_monthly_charge"] = np.where(
    dfs["tenure_months"] > 0,
    dfs["TotalCharges"] / dfs["tenure_months"],
    dfs["MonthlyCharges"]
)

# Controles y vista
print("Chequeo rápido:")
print("  Filas con prediction_date < BeginDate:", (dfs['prediction_date'] < dfs['BeginDate_dt']).sum())
print("  Nulos en prediction_date:", dfs["prediction_date"].isna().sum())
print("  Nulos en MonthlyCharges:", dfs["MonthlyCharges"].isna().sum(), 
      "| Nulos en TotalCharges:", dfs["TotalCharges"].isna().sum())
print("  Tenure meses (min/median/max):", 
      dfs["tenure_months"].min(), dfs["tenure_months"].median(), dfs["tenure_months"].max())

display(dfs[["customerID","BeginDate_dt","EndDate_dt","prediction_date","churn","tenure_months","MonthlyCharges","TotalCharges","avg_monthly_charge"]].head(8))

Chequeo rápido:
  Filas con prediction_date < BeginDate: 0
  Nulos en prediction_date: 0
  Nulos en MonthlyCharges: 0 | Nulos en TotalCharges: 11
  Tenure meses (min/median/max): 0.0 29.01 71.98


,customerID,BeginDate_dt,EndDate_dt,prediction_date,churn,tenure_months,MonthlyCharges,TotalCharges,avg_monthly_charge
0,7590-VHVEG,2020-01-01,NaT,2020-02-01,0,1.02,29.85,29.85,29.264706
1,5575-GNVDE,2017-04-01,NaT,2020-02-01,0,34.03,56.95,1889.50,55.524537
2,3668-QPYBK,2019-10-01,2019-12-01,2019-12-01,1,2.00,53.85,108.15,54.075000
3,7795-CFOCW,2016-05-01,NaT,2020-02-01,0,45.04,42.30,1840.75,40.869227
4,9237-HQITU,2019-09-01,2019-11-01,2019-11-01,1,2.00,70.70,151.65,75.825000
5,9305-CDSKC,2019-03-01,2019-11-01,2019-11-01,1,8.05,99.65,820.50,101.925466
6,1452-KIOVK,2018-04-01,NaT,2020-02-01,0,22.04,89.10,1949.40,88.448276
7,6713-OKOMC,2019-04-01,NaT,2020-02-01,0,10.05,29.75,301.90,30.039801


In [5]:
# === Selección de FEATURES y creación de splits ===

# Columnas a excluir
cols_exclude = [
    "customerID", "BeginDate", "EndDate",
    "BeginDate_dt", "EndDate_dt", "prediction_date"
]

TARGET = "churn"

# Definir FEATURES finales (todas excepto las excluidas y TARGET)
FEATURES = [c for c in dfs.columns if c not in cols_exclude + [TARGET]]

print("Total FEATURES:", len(FEATURES))
print("Ejemplo de FEATURES:", FEATURES[:10])

# Split temporal según lineamiento: antes/después de snapshot
TEMP_CUTOFF = pd.Timestamp("2020-01-01")  # training: contratos previos, valid: posteriores

train_temporal = dfs[dfs["BeginDate_dt"] < TEMP_CUTOFF].copy()
valid_temporal = dfs[dfs["BeginDate_dt"] >= TEMP_CUTOFF].copy()

print("\nSplit temporal shapes:")
print(train_temporal.shape, valid_temporal.shape)

print("\nPrevalencia churn (train/valid temporal):")
print(train_temporal[TARGET].mean().round(3), valid_temporal[TARGET].mean().round(3))

# Split estratificado (para entrenamiento real del modelo)
from sklearn.model_selection import train_test_split

train_strat, valid_strat = train_test_split(
    dfs, test_size=0.2, random_state=SEED, stratify=dfs[TARGET]
)

print("\nSplit estratificado shapes:")
print(train_strat.shape, valid_strat.shape)

print("\nPrevalencia churn (train/valid estrat):")
print(train_strat[TARGET].mean().round(3), valid_strat[TARGET].mean().round(3))

# Verificación rápida
assert TARGET in dfs.columns, "TARGET no encontrado"
assert len(FEATURES) > 0, "FEATURES vacío"

Total FEATURES: 20
Ejemplo de FEATURES: ['Type', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'InternetService']

Split temporal shapes:
(6799, 27) (244, 27)

Prevalencia churn (train/valid temporal):
0.275 0.0

Split estratificado shapes:
(5634, 27) (1409, 27)

Prevalencia churn (train/valid estrat):
0.265 0.265


In [6]:
# ===  Preprocesamiento (imputación + OHE + escalado) ===

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Identificar tipos de columnas (ya tenemos FEATURES)
cat_cols = [c for c in FEATURES if dfs[c].dtype == 'O']
num_cols = [c for c in FEATURES if c not in cat_cols]

print("Num cols:", num_cols)
print("Cat cols:", cat_cols)

# Pipelines
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Manejo de compatibilidad sklearn (sparse_output vs sparse)
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", ohe)
])

# ColumnTransformer final
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols)
    ],
    remainder="drop"
)

print("\n✅ Preprocesador creado correctamente")

Num cols: ['MonthlyCharges', 'TotalCharges', 'SeniorCitizen', 'tenure_days', 'tenure_months', 'avg_monthly_charge']
Cat cols: ['Type', 'PaperlessBilling', 'PaymentMethod', 'gender', 'Partner', 'Dependents', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']

✅ Preprocesador creado correctamente


In [7]:
# === Baseline con DummyClassifier ===

from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

def evaluate_dummy(train_df, valid_df, features=FEATURES, target=TARGET, preprocess=preprocess):
    X_train, y_train = train_df[features], train_df[target]
    X_valid, y_valid = valid_df[features], valid_df[target]

    dummy = DummyClassifier(strategy="most_frequent")  # constante: predice clase más común
    pipe = Pipeline([("prep", preprocess), ("model", dummy)])

    pipe.fit(X_train, y_train)

    # Manejo robusto de AUC: si solo hay una clase en el valid → AUC = NaN
    try:
        y_proba = pipe.predict_proba(X_valid)
        if y_proba.shape[1] == 2:
            y_score = y_proba[:, 1]
        else:
            y_score = np.zeros(len(y_valid))  # modelo degenerado
        auc = roc_auc_score(y_valid, y_score)
    except:
        auc = np.nan

    # Accuracy siempre válido
    y_pred = pipe.predict(X_valid)
    acc = accuracy_score(y_valid, y_pred)

    return auc, acc

# --- Evaluación temporal y estratificada ---
auc_temp, acc_temp = evaluate_dummy(train_temporal, valid_temporal)
auc_strat, acc_strat = evaluate_dummy(train_strat,    valid_strat)

print("Baseline Dummy Results:")
print(f" Temporal Split → AUC: {auc_temp}, Accuracy: {acc_temp:.3f}")
print(f" Stratified Split → AUC: {auc_strat:.3f}, Accuracy: {acc_strat:.3f}")

if pd.isna(auc_temp):
    print("\n⚠️  Nota: El split temporal no tiene churn en valid → AUC no definido (normal)")

Baseline Dummy Results:
 Temporal Split → AUC: nan, Accuracy: 1.000
 Stratified Split → AUC: 0.500, Accuracy: 0.735

⚠️  Nota: El split temporal no tiene churn en valid → AUC no definido (normal)


In [8]:
# ===  Modelo 1: Regresión Logística (split estratificado) ===
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report

# Pipeline: preprocess (ya creado) + modelo interpretable
logreg_clf = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", LogisticRegression(
        max_iter=2000,
        solver="lbfgs",
        class_weight="balanced",  # maneja desbalance
        n_jobs=None                # compat con sklearn 0.24.1 (sin parámetro n_jobs en LR)
    ))
])

# Entrenar en split estratificado
X_train = train_strat[FEATURES]; y_train = train_strat[TARGET]
X_valid = valid_strat[FEATURES]; y_valid = valid_strat[TARGET]

logreg_clf.fit(X_train, y_train)

# Probabilidades (robusto)
y_proba_lr = logreg_clf.predict_proba(X_valid)
if y_proba_lr.ndim == 2 and y_proba_lr.shape[1] >= 2:
    y_score_lr = y_proba_lr[:, 1]
else:
    # (muy raro con LR) fallback
    y_score_lr = np.zeros(len(y_valid), dtype=float)

# Predicción dura
y_pred_lr = logreg_clf.predict(X_valid)

# Métricas
auc_lr = roc_auc_score(y_valid, y_score_lr)
acc_lr = accuracy_score(y_valid, y_pred_lr)
cm_lr  = confusion_matrix(y_valid, y_pred_lr)

print(f"Logistic Regression (valid estrat.) -> AUC-ROC: {auc_lr:.3f} | Accuracy: {acc_lr:.3f}")
print("Confusion matrix [ [TN FP] ; [FN TP] ]:")
print(cm_lr)

# (Opcional) Reporte breve
print("\nClasification report (resumen):")
print(classification_report(y_valid, y_pred_lr, digits=3))

Logistic Regression (valid estrat.) -> AUC-ROC: 0.836 | Accuracy: 0.731
Confusion matrix [ [TN FP] ; [FN TP] ]:
[[726 309]
 [ 70 304]]

Clasification report (resumen):
              precision    recall  f1-score   support

           0      0.912     0.701     0.793      1035
           1      0.496     0.813     0.616       374

    accuracy                          0.731      1409
   macro avg      0.704     0.757     0.705      1409
weighted avg      0.802     0.731     0.746      1409



In [9]:
# === Modelo 2: HistGradientBoosting (split estratificado) ===
from sklearn.experimental import enable_hist_gradient_boosting  # compat con sklearn 0.24.1
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report

hgb_clf = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", HistGradientBoostingClassifier(
        learning_rate=0.1,
        max_iter=400,
        max_depth=None,          # deja que encuentre la complejidad óptima
        min_samples_leaf=20,
        l2_regularization=0.0,
        random_state=SEED
    ))
])

# Entrenar
X_train = train_strat[FEATURES]; y_train = train_strat[TARGET]
X_valid = valid_strat[FEATURES]; y_valid = valid_strat[TARGET]

hgb_clf.fit(X_train, y_train)

# Probabilidades y predicciones
y_proba_hgb = hgb_clf.predict_proba(X_valid)
if y_proba_hgb.ndim == 2 and y_proba_hgb.shape[1] >= 2:
    y_score_hgb = y_proba_hgb[:, 1]
else:
    y_score_hgb = np.zeros(len(y_valid), dtype=float)

y_pred_hgb = hgb_clf.predict(X_valid)

# Métricas
auc_hgb = roc_auc_score(y_valid, y_score_hgb)
acc_hgb = accuracy_score(y_valid, y_pred_hgb)
cm_hgb  = confusion_matrix(y_valid, y_pred_hgb)

print(f"HistGradientBoosting (valid estrat.) -> AUC-ROC: {auc_hgb:.3f} | Accuracy: {acc_hgb:.3f}")
print("Confusion matrix [ [TN FP] ; [FN TP] ]:")
print(cm_hgb)

print("\nClassification report:")
print(classification_report(y_valid, y_pred_hgb, digits=3))

HistGradientBoosting (valid estrat.) -> AUC-ROC: 0.891 | Accuracy: 0.844
Confusion matrix [ [TN FP] ; [FN TP] ]:
[[966  69]
 [151 223]]

Classification report:
              precision    recall  f1-score   support

           0      0.865     0.933     0.898      1035
           1      0.764     0.596     0.670       374

    accuracy                          0.844      1409
   macro avg      0.814     0.765     0.784      1409
weighted avg      0.838     0.844     0.837      1409



In [10]:
# === Selección del mejor modelo (AUC-ROC) ===
# Requiere: auc_lr, acc_lr, logreg_clf, y_score_lr
#           auc_hgb, acc_hgb, hgb_clf,   y_score_hgb
#           y_valid (valid_strat[TARGET])

results = [
    ("LogReg", auc_lr,  acc_lr,  logreg_clf, y_score_lr),
    ("HGB",    auc_hgb, acc_hgb, hgb_clf,    y_score_hgb),
]
results_sorted = sorted(results, key=lambda x: x[1], reverse=True)
best_name, best_auc, best_acc, best_model, best_scores = results_sorted[0]

print("=== Mejor modelo por AUC-ROC ===")
print(f"Modelo: {best_name}")
print(f"AUC-ROC: {best_auc:.3f} | Accuracy: {best_acc:.3f}")

=== Mejor modelo por AUC-ROC ===
Modelo: HGB
AUC-ROC: 0.891 | Accuracy: 0.844


In [11]:
# === Threshold operativo (Youden J y F1) ===
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score, f1_score

y_true = valid_strat[TARGET].values
y_score = best_scores  # del mejor modelo

# Youden J
fpr, tpr, thr = roc_curve(y_true, y_score)
youden = tpr - fpr
thr_youden = thr[youden.argmax()]

# Máx F1
prec, rec, thr_pr = precision_recall_curve(y_true, y_score)
f1_vals = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
thr_f1 = thr_pr[f1_vals.argmax()]

def eval_at(thr):
    pred = (y_score >= thr).astype(int)
    acc  = accuracy_score(y_true, pred)
    f1v  = f1_score(y_true, pred, zero_division=0)
    cm   = confusion_matrix(y_true, pred)
    return acc, f1v, cm

acc_y, f1_y, cm_y = eval_at(thr_youden)
acc_f, f1_f, cm_f = eval_at(thr_f1)

print(f"Threshold Youden J: {thr_youden:.4f} | Acc: {acc_y:.3f} | F1: {f1_y:.3f}")
print(cm_y)
print(f"\nThreshold máx F1 : {thr_f1:.4f} | Acc: {acc_f:.3f} | F1: {f1_f:.3f}")
print(cm_f)

Threshold Youden J: 0.1357 | Acc: 0.792 | F1: 0.678
[[807 228]
 [ 65 309]]

Threshold máx F1 : 0.2975 | Acc: 0.836 | F1: 0.700
[[908 127]
 [104 270]]


In [12]:
# === Interpretabilidad (Permutation Importance) ===
from sklearn.inspection import permutation_importance

X_valid = valid_strat[FEATURES]
y_valid = valid_strat[TARGET]

r = permutation_importance(
    best_model, X_valid, y_valid,
    n_repeats=10, random_state=SEED, scoring="roc_auc"
)

imp_df = (
    pd.DataFrame({
        "feature": FEATURES,
        "importance_mean": r.importances_mean,
        "importance_std":  r.importances_std
    })
    .sort_values("importance_mean", ascending=False)
)

print("Top 15 características más influyentes (Permutation Importance, por AUC):")
display(imp_df.head(15))

Top 15 características más influyentes (Permutation Importance, por AUC):


,feature,importance_mean,importance_std
17,tenure_days,0.254513,0.009328
0,Type,0.077115,0.006799
4,TotalCharges,0.021963,0.002277
3,MonthlyCharges,0.016410,0.004808
9,InternetService,0.001635,0.000626
13,TechSupport,0.001466,0.001356
14,StreamingTV,0.001003,0.001113
8,Dependents,0.000918,0.001145
2,PaymentMethod,0.000845,0.002155
15,StreamingMovies,0.000629,0.000420


In [13]:
# === Segmentación por variables clave ===

# Puntajes (probabilidad) del mejor modelo sobre valid estratificado
valid_view = valid_strat.copy()
valid_view["score"] = best_model.predict_proba(valid_view[FEATURES])[:, 1]

# 1) Por tipo de contrato
if "Type" in valid_view.columns:
    seg_type = valid_view.groupby("Type")["score"].agg(["mean","count"]).rename(columns={"mean":"riesgo_prom","count":"n"})
    print("Riesgo promedio por Type:")
    display(seg_type.sort_values("riesgo_prom", ascending=False))

# 2) Por tecnología de internet
if "InternetService" in valid_view.columns:
    seg_net = valid_view.groupby("InternetService")["score"].agg(["mean","count"]).rename(columns={"mean":"riesgo_prom","count":"n"})
    print("Riesgo promedio por InternetService:")
    display(seg_net.sort_values("riesgo_prom", ascending=False))

# 3) Por cuartiles de MonthlyCharges
if "MonthlyCharges" in valid_view.columns:
    valid_view["mc_bin"] = pd.qcut(valid_view["MonthlyCharges"], q=4, duplicates="drop")
    seg_mc = valid_view.groupby("mc_bin")["score"].agg(["mean","count"]).rename(columns={"mean":"riesgo_prom","count":"n"})
    print("Riesgo promedio por cuartiles de MonthlyCharges:")
    display(seg_mc.sort_values("riesgo_prom", ascending=False))

Riesgo promedio por Type:


,riesgo_prom,n
Type,,
Month-to-month,0.397531,773
One year,0.063218,300
Two year,0.010428,336


Riesgo promedio por InternetService:


,riesgo_prom,n
InternetService,,
Fiber optic,0.395681,613
DSL,0.140215,484


Riesgo promedio por cuartiles de MonthlyCharges:


,riesgo_prom,n
mc_bin,,
"(69.95, 89.45]",0.370860,351
"(89.45, 116.8]",0.286300,350
"(35.2, 69.95]",0.186786,355
"(18.249, 35.2]",0.093694,353


In [14]:
# === Top riesgo y guardado de artefactos ===
import joblib

cols_exist = [c for c in ["customerID","Type","InternetService","MonthlyCharges"] if c in valid_view.columns]
top_risk = valid_view[cols_exist + ["score"]].sort_values("score", ascending=False).head(50)

print("Top 50 clientes en riesgo (valid estratificado):")
display(top_risk)

# Guardar modelo y lista
joblib.dump(best_model, ARTIFACTS / f"best_model_{best_name}.joblib")
top_risk.to_csv(ARTIFACTS / "top_risk_valid_strat.csv", index=False)
print("\nGuardado:", ARTIFACTS / f"best_model_{best_name}.joblib")
print("Guardado:", ARTIFACTS / "top_risk_valid_strat.csv")

Top 50 clientes en riesgo (valid estratificado):


,customerID,Type,InternetService,MonthlyCharges,score
6623,9248-OJYKK,Month-to-month,Fiber optic,76.45,0.999976
2194,2514-GINMM,Month-to-month,Fiber optic,79.50,0.999958
809,1820-TQVEV,Month-to-month,Fiber optic,69.55,0.999957
1334,1768-ZAIFU,Month-to-month,DSL,25.20,0.999944
6509,0616-ATFGB,Month-to-month,DSL,25.05,0.999907
1807,6910-HADCM,Month-to-month,Fiber optic,76.35,0.999891
2464,2609-IAICY,Month-to-month,Fiber optic,77.15,0.999862
2517,5186-SAMNZ,Month-to-month,Fiber optic,69.50,0.999860
1739,9804-ICWBG,Month-to-month,Fiber optic,69.90,0.999812
5826,9507-EXLTT,Month-to-month,Fiber optic,79.35,0.999741



Guardado: ..\artifacts\best_model_HGB.joblib
Guardado: ..\artifacts\top_risk_valid_strat.csv


In [15]:
# === Top riesgo y guardado de artefactos (REEMPLAZO) ===
import joblib

# Construimos 'valid_view' sin depender de variables previas de proba
valid_view = valid_strat.copy()
valid_view["score"] = best_model.predict_proba(valid_view[FEATURES])[:, 1]

cols_exist = [c for c in ["customerID","Type","InternetService","MonthlyCharges"] if c in valid_view.columns]
top_risk = valid_view[cols_exist + ["score"]].sort_values("score", ascending=False).head(50)

print("Top 50 clientes en riesgo (valid estratificado):")
display(top_risk)

# Guardar modelo y lista
joblib.dump(best_model, ARTIFACTS / f"best_model_{best_name}.joblib")
top_risk.to_csv(ARTIFACTS / "top_risk_valid_strat.csv", index=False)
print("\nGuardado:", ARTIFACTS / f"best_model_{best_name}.joblib")
print("Guardado:", ARTIFACTS / "top_risk_valid_strat.csv")

Top 50 clientes en riesgo (valid estratificado):


,customerID,Type,InternetService,MonthlyCharges,score
6623,9248-OJYKK,Month-to-month,Fiber optic,76.45,0.999976
2194,2514-GINMM,Month-to-month,Fiber optic,79.50,0.999958
809,1820-TQVEV,Month-to-month,Fiber optic,69.55,0.999957
1334,1768-ZAIFU,Month-to-month,DSL,25.20,0.999944
6509,0616-ATFGB,Month-to-month,DSL,25.05,0.999907
1807,6910-HADCM,Month-to-month,Fiber optic,76.35,0.999891
2464,2609-IAICY,Month-to-month,Fiber optic,77.15,0.999862
2517,5186-SAMNZ,Month-to-month,Fiber optic,69.50,0.999860
1739,9804-ICWBG,Month-to-month,Fiber optic,69.90,0.999812
5826,9507-EXLTT,Month-to-month,Fiber optic,79.35,0.999741



Guardado: ..\artifacts\best_model_HGB.joblib
Guardado: ..\artifacts\top_risk_valid_strat.csv


In [16]:
# === Threshold operativo (Youden J y F1) — REEMPLAZO COMPLETO ===
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score, f1_score
import numpy as np

# Definimos y_true y y_score sin depender de variables previas
y_true  = valid_strat[TARGET].values
y_score = best_model.predict_proba(valid_strat[FEATURES])[:, 1]  # prob. de churn

# --- Threshold por Youden J (maximiza tpr - fpr) ---
fpr, tpr, thr = roc_curve(y_true, y_score)
youden = tpr - fpr
thr_youden = thr[int(np.argmax(youden))]

# --- Threshold por máximo F1 (alineado con thresholds de PR) ---
prec, rec, thr_pr = precision_recall_curve(y_true, y_score)
# precision_recall_curve devuelve N+1 puntos; los thresholds son de tamaño N
# Calculamos F1 para los puntos que sí tienen threshold:
f1_vals = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
idx_f1  = int(np.argmax(f1_vals))
thr_f1  = thr_pr[idx_f1]

def eval_at(th):
    pred = (y_score >= th).astype(int)
    acc  = accuracy_score(y_true, pred)
    f1v  = f1_score(y_true, pred, zero_division=0)
    cm   = confusion_matrix(y_true, pred)
    return acc, f1v, cm

acc_y, f1_y, cm_y = eval_at(thr_youden)
acc_f, f1_f, cm_f = eval_at(thr_f1)

print(f"Threshold Youden J: {thr_youden:.4f} | Acc: {acc_y:.3f} | F1: {f1_y:.3f}")
print(cm_y)
print()
print(f"Threshold máx F1 : {thr_f1:.4f} | Acc: {acc_f:.3f} | F1: {f1_f:.3f}")
print(cm_f)

Threshold Youden J: 0.1357 | Acc: 0.792 | F1: 0.678
[[807 228]
 [ 65 309]]

Threshold máx F1 : 0.2975 | Acc: 0.836 | F1: 0.700
[[908 127]
 [104 270]]


In [17]:

top_risk.to_csv(ARTIFACTS / "top50_churn_risk_valid.csv", index=False)
print("Archivo guardado:", ARTIFACTS / "top50_churn_risk_valid.csv")


Archivo guardado: ..\artifacts\top50_churn_risk_valid.csv


In [18]:
# === Exportar artefactos finales del modelo ===
import joblib
from pathlib import Path

# Crear carpeta artifacts si no existe
ARTIFACTS.mkdir(parents=True, exist_ok=True)

# Guardar modelo entrenado
model_path = ARTIFACTS / f"best_model_{best_name}.joblib"
joblib.dump(best_model, model_path)

# Guardar top riesgo
csv_path = ARTIFACTS / "top_50_risk_valid_strat.csv"
top_risk.to_csv(csv_path, index=False)

print("✅ Artefactos guardados")
print("Modelo:", model_path)
print("Top riesgo:", csv_path)

✅ Artefactos guardados
Modelo: ..\artifacts\best_model_HGB.joblib
Top riesgo: ..\artifacts\top_50_risk_valid_strat.csv


In [19]:
# === Resumen final del modelo (métricas + umbrales + matrices) ===
import json
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, precision_recall_curve, f1_score
import numpy as np

# Probabilidades y etiquetas en valid estratificado (cierre consistente)
y_true  = valid_strat[TARGET].values
y_score = best_model.predict_proba(valid_strat[FEATURES])[:, 1]
y_pred  = best_model.predict(valid_strat[FEATURES])

# Métricas a umbral 0.5 (informativo)
auc_05 = roc_auc_score(y_true, y_score)
acc_05 = accuracy_score(y_true, y_pred)
cm_05  = confusion_matrix(y_true, y_pred)

# Threshold Youden J
fpr, tpr, thr = roc_curve(y_true, y_score)
youden = tpr - fpr
thr_youden = thr[int(np.argmax(youden))]
pred_youden = (y_score >= thr_youden).astype(int)
acc_youden  = accuracy_score(y_true, pred_youden)
f1_youden   = f1_score(y_true, pred_youden, zero_division=0)
cm_youden   = confusion_matrix(y_true, pred_youden)

# Threshold máx F1
prec, rec, thr_pr = precision_recall_curve(y_true, y_score)
f1_vals = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
idx_f1  = int(np.argmax(f1_vals))
thr_f1  = thr_pr[idx_f1]
pred_f1 = (y_score >= thr_f1).astype(int)
acc_f1  = accuracy_score(y_true, pred_f1)
f1_f1   = f1_score(y_true, pred_f1, zero_division=0)
cm_f1   = confusion_matrix(y_true, pred_f1)

resumen = {
    "best_model": str(best_name),
    "valid_strat": {
        "auc": round(auc_05, 3),
        "accuracy@0.5": round(acc_05, 3),
        "cm@0.5": cm_05.tolist()
    },
    "thresholds": {
        "youden": {
            "thr": round(float(thr_youden), 4),
            "accuracy": round(acc_youden, 3),
            "f1": round(f1_youden, 3),
            "cm": cm_youden.tolist()
        },
        "f1_max": {
            "thr": round(float(thr_f1), 4),
            "accuracy": round(acc_f1, 3),
            "f1": round(f1_f1, 3),
            "cm": cm_f1.tolist()
        }
    }
}

print(json.dumps(resumen, indent=2))

{
  "best_model": "HGB",
  "valid_strat": {
    "auc": 0.891,
    "accuracy@0.5": 0.844,
    "cm@0.5": [
      [
        966,
        69
      ],
      [
        151,
        223
      ]
    ]
  },
  "thresholds": {
    "youden": {
      "thr": 0.1357,
      "accuracy": 0.792,
      "f1": 0.678,
      "cm": [
        [
          807,
          228
        ],
        [
          65,
          309
        ]
      ]
    },
    "f1_max": {
      "thr": 0.2975,
      "accuracy": 0.836,
      "f1": 0.7,
      "cm": [
        [
          908,
          127
        ],
        [
          104,
          270
        ]
      ]
    }
  }
}


# Conclusiones y Recomendaciones

## 1) Resumen ejecutivo
Entrenamos un modelo de **churn** para Interconnect. En validación estratificada alcanzó **AUC-ROC = 0.891** y **accuracy = 0.844** a umbral 0.5, superando la meta (≥ 0.88). El modelo seleccionado es **HistGradientBoosting** con un pipeline de preprocesamiento (imputación, OHE, escalado).

> **Nota sobre split temporal:** la validación temporal quedó con **clase única (0% churn)**; por ello la métrica oficial se reporta en el split estratificado. Se documentó en el notebook.

## 2) Principales impulsores (drivers)
Las variables más influyentes fueron **tenure** (antigüedad), **tipo de contrato** (month-to-month eleva riesgo), **tecnología de internet** y **cargos mensuales**. Clientes relativamente **nuevos**, con **contrato mes a mes** y **cargos altos** presentan mayor propensión a cancelar.

## 3) Umbral operativo recomendado
Se evaluaron dos criterios:
- **Youden J (0.1357):** mayor recall, conveniente si la pérdida por fuga es alta y el costo por contacto es bajo.
- **Máx F1 (0.2975):** mejor equilibrio precisión/recall, conveniente si el costo por contacto importa más.

**Recomendación:** iniciar campañas con **F1-máx (0.2975)** y correr una **prueba A/B** con **Youden** para comparar costo por retención y lift.

## 4) Segmentos prioritarios
- **Contrato month-to-month:** ofrecer permanencia (1–2 años) con beneficio claro.
- **Tecnología (p. ej. fibra):** revisar pain points (instalación, soporte, facturación) si muestran riesgo ↑.
- **Cargos altos:** diseñar empaquetados con descuentos o servicios de valor (Soporte, Seguridad, Backup).

## 5) Entregables operativos
- **Modelo:** `artifacts/best_model_HGB.joblib`
- **Top 50 en riesgo (valid):** `artifacts/top_50_risk_valid_strat.csv` con `customerID`, `Type`, `InternetService`, `MonthlyCharges`, `score`.
- **Notebook:** documenta momento de predicción, ingeniería, splits, métricas y umbrales.

## 6) Riesgos y siguientes pasos
- **Calibración:** evaluar **Brier score** y considerar **CalibratedClassifierCV** (isotónica o sigmoide).
- **Monitoreo:** chequear **drift** (features/score) y reentrenar trimestral o si AUC baja >2–3 p.p.
- **Economía:** traducir métricas a **impacto de negocio** (costo por contacto, tasa de aceptación, valor del cliente retenido).

In [22]:
# === Calibración: CalibratedClassifierCV + Brier score ===
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import brier_score_loss, roc_auc_score, accuracy_score

# Tomamos el best_model (HGB) ya entrenado y lo calibramos sobre el train_strat con validación interna.
# Nota: Para ser muy riguroso, separarías un fold de calibración. Aquí usamos CV=3 para rapidez.
calib = CalibratedClassifierCV(estimator=best_model.named_steps["clf"], method="isotonic", cv=3)



# Reentrenamos un pipeline: mismo preprocess + calibrador
calib_pipe = Pipeline([
    ("prep", best_model.named_steps["prep"]),
    ("calib", calib)
])

calib_pipe.fit(train_strat[FEATURES], train_strat[TARGET])

# Evaluación en valid_strat
y_true  = valid_strat[TARGET].values
y_score = calib_pipe.predict_proba(valid_strat[FEATURES])[:, 1]
y_pred  = calib_pipe.predict(valid_strat[FEATURES])

auc_c   = roc_auc_score(y_true, y_score)
acc_c   = accuracy_score(y_true, y_pred)
brier_c = brier_score_loss(y_true, y_score)

print(f"Calibrated (isotonic) → AUC: {auc_c:.3f} | Acc: {acc_c:.3f} | Brier: {brier_c:.4f}")

Calibrated (isotonic) → AUC: 0.885 | Acc: 0.837 | Brier: 0.1128


In [24]:
# === Guardar modelo calibrado (pipeline completo) ===
import joblib

calib_path = ARTIFACTS / "best_model_HGB_calibrated_isotonic.joblib"
joblib.dump(calib_pipe, calib_path)
print("✅ Modelo calibrado guardado en:", calib_path)


✅ Modelo calibrado guardado en: ..\artifacts\best_model_HGB_calibrated_isotonic.joblib


In [26]:
# === Thresholds con modelo calibrado ===
from sklearn.metrics import roc_curve, precision_recall_curve, accuracy_score, f1_score, confusion_matrix
import numpy as np

y_true   = valid_strat[TARGET].values
y_scoreC = calib_pipe.predict_proba(valid_strat[FEATURES])[:, 1]

# Youden J
fpr, tpr, thr = roc_curve(y_true, y_scoreC)
youden = tpr - fpr
thr_y  = thr[int(np.argmax(youden))]

# Máx F1
prec, rec, thr_pr = precision_recall_curve(y_true, y_scoreC)
f1_vals = 2 * (prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
thr_f   = thr_pr[int(np.argmax(f1_vals))]

def eval_at(th):
    pred = (y_scoreC >= th).astype(int)
    acc  = accuracy_score(y_true, pred)
    f1v  = f1_score(y_true, pred, zero_division=0)
    cm   = confusion_matrix(y_true, pred)
    return acc, f1v, cm

acc_y, f1_y, cm_y = eval_at(thr_y)
acc_f, f1_f, cm_f = eval_at(thr_f)

print(f"Calibrated — Threshold Youden J: {thr_y:.4f} | Acc: {acc_y:.3f} | F1: {f1_y:.3f}")
print(cm_y)
print()
print(f"Calibrated — Threshold máx F1 : {thr_f:.4f} | Acc: {acc_f:.3f} | F1: {f1_f:.3f}")
print(cm_f)

Calibrated — Threshold Youden J: 0.2301 | Acc: 0.776 | F1: 0.666
[[780 255]
 [ 60 314]]

Calibrated — Threshold máx F1 : 0.3322 | Acc: 0.827 | F1: 0.687
[[897 138]
 [106 268]]


In [28]:
# === Top 50 riesgo con modelo calibrado ===
valid_calib = valid_strat.copy()
valid_calib["score"] = calib_pipe.predict_proba(valid_calib[FEATURES])[:, 1]

cols_exist = [c for c in ["customerID","Type","InternetService","MonthlyCharges"] if c in valid_calib.columns]
top50_calib = valid_calib[cols_exist + ["score"]].sort_values("score", ascending=False).head(50)

display(top50_calib)

# Guardar lista calibrada
csv_calib = ARTIFACTS / "top_50_risk_valid_strat_calibrated.csv"
top50_calib.to_csv(csv_calib, index=False)
print("✅ Guardado:", csv_calib)

,customerID,Type,InternetService,MonthlyCharges,score
6388,1963-VAUKV,Month-to-month,NaN,20.40,1.000000
2397,3722-WPXTK,Month-to-month,Fiber optic,88.35,1.000000
1807,6910-HADCM,Month-to-month,Fiber optic,76.35,1.000000
6190,2867-UIMSS,Month-to-month,Fiber optic,80.50,1.000000
3090,2386-LAHRK,Month-to-month,DSL,53.50,1.000000
2526,5766-ZJYBB,Month-to-month,NaN,19.40,1.000000
5100,9940-RHLFB,Month-to-month,Fiber optic,75.30,1.000000
2631,6861-XWTWQ,Month-to-month,Fiber optic,99.25,1.000000
2517,5186-SAMNZ,Month-to-month,Fiber optic,69.50,1.000000
2194,2514-GINMM,Month-to-month,Fiber optic,79.50,1.000000


✅ Guardado: ..\artifacts\top_50_risk_valid_strat_calibrated.csv


Calibración de probabilidades. Se aplico CalibratedClassifierCV (isotónica) sobre el mejor modelo (HGB). El AUC pasó de 0.891 a 0.885 (cambio marginal), mientras que el Brier score mejoró a 0.1128, indicando probabilidades mejor calibradas. Para campañas de retención basadas en umbrales y priorización por score, recomendamos usar el modelo calibrado y el umbral F1-máx como punto de partida, dejando Youden como alternativa en una prueba A/B.

Resultado clave
El mejor modelo fue HistGradientBoosting con calibración isotónica, alcanzando un AUC-ROC de 0.885 y una exactitud de 0.837. La calibración mejoró la calidad de las probabilidades (Brier score = 0.1128), lo que permite priorizar clientes por riesgo de cancelación y definir umbrales operativos para campañas de retención.